<p style="text-align:center;">
<img src="https://github.com/digital-futures-academy/DataScienceMasterResources/blob/main/Resources/datascience-notebook-header.png?raw=true"
     alt="DigitalFuturesLogo"
     style="float: center; margin-right: 10px;" />
</p>

# Swan Consulting Workbook
### <i>By Team White Swan</i>

### Table Of Contents:
* [General Properties](#1)
* [Exploratory Data Analysis (EDA)](#2)
* [Train-test-split](#3)


The aim of this workbook is to .....

In [21]:
## Importing Libraries
 #For Handling and Visualising Data
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
 # For Modelling
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier , ExtraTreesClassifier # Decison Tree Model
from sklearn.linear_model import LogisticRegression # Logistic Regression Model
import statsmodels as sm

# For measuring our models performance
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score

In [26]:
## Read in Data
df = pd.read_csv('Swan_Project_Data.csv')

## **1** General properties <a class="anchor" id="1"></a>

In [35]:
## Check first few rows
 # Set option to display all columns
pd.set_option('display.max_columns', None)
df.head(3)

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,Moved


In [216]:
## Check shape
df.shape

(2864, 21)

In [218]:
## Check columns
df.columns

Index(['Country', 'Region', 'Year', 'Infant_deaths', 'Under_five_deaths',
       'Adult_mortality', 'Alcohol_consumption', 'Hepatitis_B', 'Measles',
       'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed',
       'Economy_status_Developing', 'Life_expectancy'],
      dtype='object')

In [220]:
## Check data types
df.dtypes

Country                         object
Region                          object
Year                             int64
Infant_deaths                  float64
Under_five_deaths              float64
Adult_mortality                float64
Alcohol_consumption            float64
Hepatitis_B                      int64
Measles                          int64
BMI                            float64
Polio                            int64
Diphtheria                       int64
Incidents_HIV                  float64
GDP_per_capita                   int64
Population_mln                 float64
Thinness_ten_nineteen_years    float64
Thinness_five_nine_years       float64
Schooling                      float64
Economy_status_Developed         int64
Economy_status_Developing        int64
Life_expectancy                float64
dtype: object

## **2** EDA <a class="anchor" id="2"></a>

### **2.1** Summary Statistics

First, check the summary statistics to get a feel of the dataset.

In [262]:
# Check summary statistics
train_df.describe()

,Year,Infant_deaths,Under_five_deaths,Adult_mortality,Alcohol_consumption,Hepatitis_B,Measles,BMI,Polio,Diphtheria,Incidents_HIV,GDP_per_capita,Population_mln,Thinness_ten_nineteen_years,Thinness_five_nine_years,Schooling,Economy_status_Developed,Economy_status_Developing,Life_expectancy
count,2291.000000,2291.000000,2291.000000,2291.000000,2291.000000,2291.000000,2291.000000,2291.000000,2291.000000,2291.000000,2291.000000,2291.000000,2291.00000,2291.000000,2291.000000,2291.000000,2291.000000,2291.000000,2291.000000
mean,2007.450022,30.482933,42.976910,192.291228,4.826151,84.350502,77.527717,25.051113,86.527281,86.253601,0.914588,11493.591445,36.53900,4.829027,4.892449,7.637320,0.205151,0.794849,68.852248
std,4.654388,27.815476,44.776425,115.246120,3.989247,16.078459,18.641422,2.186557,15.164701,15.659472,2.459888,16900.130926,136.96815,4.430364,4.540586,3.170865,0.403900,0.403900,9.424849
min,2000.000000,1.800000,2.300000,49.998000,0.000000,14.000000,12.000000,19.800000,8.000000,16.000000,0.010000,148.000000,0.08000,0.100000,0.100000,1.100000,0.000000,0.000000,39.400000
25%,2003.000000,8.150000,9.700000,106.374250,1.205000,79.000000,64.000000,23.300000,82.000000,81.000000,0.080000,1422.500000,2.06500,1.600000,1.600000,5.000000,0.000000,1.000000,62.900000
50%,2007.000000,19.500000,22.900000,163.946000,3.990000,89.000000,83.000000,25.600000,93.000000,93.000000,0.150000,4221.000000,7.55000,3.300000,3.300000,7.800000,0.000000,1.000000,71.300000
75%,2012.000000,47.800000,65.950000,246.226000,7.825000,96.000000,93.000000,26.500000,97.000000,97.000000,0.430000,12628.500000,23.56000,7.200000,7.300000,10.300000,0.000000,1.000000,75.500000
max,2015.000000,138.100000,224.900000,719.360500,17.750000,99.000000,99.000000,32.100000,99.000000,99.000000,21.680000,112418.000000,1379.86000,27.700000,28.600000,14.100000,1.000000,1.000000,83.800000


><b>Insights</b>:\
    * The dataset contains data contain 
    * 

### Nulls

In [42]:
## Handling missing values

def null_vals(dataframe):
    '''Function to show both number of nulls and the percentage of nulls in the whole column'''
    null_vals = dataframe.isnull().sum() # How many nulls in each column
    total_cnt = len(dataframe) # Total entries in the dataframe
    null_vals = pd.DataFrame(null_vals,columns=['null']) # Put the number of nulls in a single dataframe
    null_vals['percent'] = round((null_vals['null']/total_cnt)*100,3) # Round how many nulls are there, as %, of the df
    
    return null_vals.sort_values('percent', ascending=False) #Return the number and percentage of missing values, order by percentage descending

null_vals(df) #Display the table

,null,percent
Churn Reason,5174,73.463
Internet Service,0,0.000
Churn Value,0,0.000
Churn Label,0,0.000
Total Charges,0,0.000
Monthly Charges,0,0.000
Payment Method,0,0.000
Paperless Billing,0,0.000
Contract,0,0.000
Streaming Movies,0,0.000


## **3** Train-test-split <a class="anchor" id="3"></a>